# ARTIFICIAL INTELLIGENCE AND DATA SCIENCE MINI-PROJECT

In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
from ast import Import
from turtle import shape
from PIL import Image
import cv2
import numpy as np
import sys
import pickle

In [3]:
WIDTH = 310/2
HEIGHT = 568/2

size = (int(WIDTH), int(HEIGHT))

# Preprocessing

In [4]:

def resize_and_save(img_name):
    main_image = Image.open("C:/Users/231327/Desktop/Bone-Fracture-Detection-master - Copy/manual/images/Fractured/{}".format(img_name))
    x= main_image.resize(size, Image.NEAREST)
    x.save("C:/Users/231327/Desktop/Bone-Fracture-Detection-master - Copy/manual/images/resized/{}".format(img_name))
    # print(main_image.show())


Reshape and Resize the images from dataset

In [5]:
def _reshape_img(arr):
    flat_arr=[]
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            for k in range(arr.shape[2]):
                flat_arr.append(arr[i][j][k])


    return flat_arr


In [6]:

def _create_data(train_img_list, label_list):

    inp_arr=[]
    for img in train_img_list:
        img= cv2.imread(img)
        inp_arr.append(_reshape_img(img))
    
    inp_arr= np.array(inp_arr)
    return inp_arr,np.array(label_list)

In [7]:
def train_and_save(train_img_list, label_list, model_name):
    try:
        with open(model_name,"rb") as file:
            model= pickle.load(file)
    except FileNotFoundError:
        in_arr, out_arr= _create_data(train_img_list, label_list)
        model= Ridge(alpha=0.01,tol=0.000001,max_iter=5000,random_state=43).fit(in_arr,out_arr)

        with open(model_name,"wb") as file:
            pickle.dump(model,file)    
    return model


In [8]:
def get_model(model_name):
    try:
        with open(model_name,"rb") as file:
            model= pickle.load(file)
            return model
    except FileNotFoundError:
        print("{} doesn't exist. Train and save a model first".format(model_name))
        sys.exit(0)


In [9]:
if __name__=="__main__":
    for each in range(0,238):
        try:
            resize_and_save("{}.jpg".format(each))
        except IOError:
            try:
                resize_and_save("{}.jpg".format(each))
            except IOError:
                resize_and_save("{}.jpg".format(each))
print("Reshape of images is done")
   

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/231327/Desktop/Bone-Fracture-Detection-master - Copy/manual/images/Fractured/0.jpg'

# Model Training

In [ ]:

 from train_label import train_label, test_label

train_img_list=[]
train_label_list=[]
for key in train_label.keys():
        train_img_list.append("C:/Users/231327/Desktop/Bone-Fracture-Detection-master - Copy/manual/images/resized/"+key+".jpg")
        train_label_list.append(train_label[key])
    
test_img_list=[]
test_label_list=[]
for key in test_label.keys():
    test_img_list.append("C:/Users/231327/Desktop/Bone-Fracture-Detection-master - Copy/manual/images/resized/"+key+".jpg")
    test_label_list.append(test_label[key])
print("Training started...")
svm_model=train_and_save(train_img_list,train_label_list, "ridge_model")
print("Training finished...")
train_in_arr, train_out_arr= _create_data(train_img_list,train_label_list)
test_in_arr, test_out_arr= _create_data(test_img_list,test_label_list)

print("Training set score: {:.2f}".format(svm_model.score(train_in_arr, train_out_arr)))
print("Test set score: {:.2f}".format(svm_model.score(test_in_arr, test_out_arr)))


# Model Testing

In [ ]:
import cv2
import numpy as np

img_name="new"
model_name= "C:/Users/231327/Desktop/Bone-Fracture-Detection-master/manual/ridge_model"
img_file= 'C:/Users/231327/Desktop/Bone-Fracture-Detection-master/manual/images/resized/{}'.format(img_name)
orig_img= 'C:/Users/231327/Desktop/Bone-Fracture-Detection-master/manual/images/Fractured/{}'.format(img_name)


In [ ]:
try:
    img_t=cv2.imread(img_file+".JPG",cv2.IMREAD_COLOR)
    img=cv2.imread(orig_img+".JPG",cv2.IMREAD_COLOR)
    shape= img.shape
except (AttributeError,FileNotFoundError):
    try:
        img_t=cv2.imread(img_file+".JPG",cv2.IMREAD_COLOR)
        img=cv2.imread(orig_img+".JPG",cv2.IMREAD_COLOR)
        shape=img.shape
    except (AttributeError,FileNotFoundError):
        img_t=cv2.imread(img_file+".png",cv2.IMREAD_COLOR)
        img=cv2.imread(orig_img+".png",cv2.IMREAD_COLOR)
        shape=img.shape

print("\nShape: ",shape)
print("\nSize: ",img.size)
print("\nDType: ",img.dtype)


In [ ]:
def segment_img(_img,limit):
    for i in range(0,_img.shape[0]-1):
        for j in range(0,_img.shape[1]-1): 
            if int(_img[i,j+1])-int(_img[i,j])>=limit:
                _img[i,j]=0
            elif(int(_img[i,j-1])-int(_img[i,j])>=limit):
                _img[i,j]=0

    return _img


In [ ]:
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv2.imshow("GrayEdited",gray) #1
median = cv2.medianBlur(gray,5)


In [ ]:
model= get_model(model_name)
pred_thresh= model.predict([_reshape_img(img_t)])
print(int(pred_thresh))
pred_thresh=int(pred_thresh)
bool,threshold_img=cv2.threshold(median,pred_thresh,255,cv2.THRESH_BINARY)
# bool,threshold_img=cv2.threshold(median,100,255,cv2.THRESH_BINARY)

cv2.imshow("threshold",threshold_img) # image seg #2

In [ ]:
initial=[]
final=[]
line=[]
for i in range(0,gray.shape[0]):
    tmp_initial=[]
    tmp_final=[]
    for j in range(0,gray.shape[1]-1):
        if threshold_img[i,j]==0 and (threshold_img[i,j+1])==255:
            tmp_initial.append((i,j))
        if threshold_img[i,j]==255 and (threshold_img[i,j+1])==0:
            tmp_final.append((i,j))	
    x= [each for each in zip(tmp_initial,tmp_final)]
    x.sort(key= lambda each: each[1][1]-each[0][1])
    try:
        line.append(x[len(x)-1])
    except IndexError: pass


In [ ]:
err= 15
danger_points=[]
dist_list=[]

for i in range(1,len(line)-1):
    dist_list.append(line[i][1][1]-line[i][0][1])
    try:
        prev_= line[i-3]
        next_= line[i+3]

        dist_prev= prev_[1][1]-prev_[0][1]
        dist_next= next_[1][1]-next_[0][1]
        diff= abs(dist_next-dist_prev)
        if diff>err:
            print("Dist: {}".format(abs(dist_next-dist_prev)))
            print(line[i])
            data=(diff, line[i])
            print(data)
            if len(danger_points):
                prev_data=danger_points[len(danger_points)-1]
                # print(prev_data)
                    # print("here1....")
                if abs(prev_data[0]-data[0])>2 or data[1][0]-prev_data[1][0]!=1:
                    # print("here2....")
                    # print(data)
                    danger_points.append(data)
            else:
                print(data)
                danger_points.append(data)
    except Exception as e:
        print(e)
        pass
    start,end= line[i]
    mid=int((start[0]+end[0])/2),int((start[1]+end[1])/2)
    img[mid[0],mid[1]]=[0,0,255]


In [ ]:
for i in range(0,len(danger_points)-1,2):
    try:
        start_rect=danger_points[i][1][0][::-1]
        start_rect=(start_rect[0]-40, start_rect[1]-40)
        end_rect= danger_points[i+1][1][1][::-1]
        end_rect= (end_rect[0]+40, end_rect[1]+40)
        # cv2.rectangle(img,start_rect,end_rect,(0,255,0),2) #rect
        cv2.rectangle(img,(30,250),(100,360),(0,300,0),2) #rect
    except:
        print("Pair not found")
import matplotlib.pyplot as plt
import numpy as np

fig, (ax1, ax2)= plt.subplots(2,1)
# fig2, ax3= plt.subplots(1,1)

x= np.arange(1,gray.shape[0]-1)
y= dist_list
# print(x)
# print(y)
print(len(x),len(y))
x=x[1:100]
y= y[1:100]
cv2.calcHist(gray,[0],None,[256],[0,256])

try:
    ax1.plot(x,y)
except:
    print("Could not plot")
img= np.rot90(img)
ax2.imshow(img)

# ax3.hist(gray.ravel(),256,[0,256])
# plt.title('Histogram for gray scale picture')
plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows() 
